# Deep Reinforcement Learning Laboratory

In this laboratory session we will work on getting more advanced versions of Deep Reinforcement Learning algorithms up and running. Deep Reinforcement Learning is **hard**, and getting agents to stably train can be frustrating and requires quite a bit of subtlety in analysis of intermediate results. We will start by refactoring (a bit) my implementation of `REINFORCE` on the [Cartpole environment](https://gymnasium.farama.org/environments/classic_control/cart_pole/). 

## Exercise 1: Improving my `REINFORCE` Implementation (warm up)

In this exercise we will refactor a bit and improve some aspects of my `REINFORCE` implementation. 

**First Things First**: Spend some time playing with the environment to make sure you understand how it works.

In [1]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Instantiate a rendering and a non-rendering environment.
env_render = gym.make('CartPole-v1', render_mode='human')
env = gym.make('CartPole-v1')

CartPole è un carrello che può muoversi a sinistra o a destra su un binario, sopra il carello è montato un pali collegato con una cerniera alla base, l'agente deve bilanciare il palo muovendo il carello 
Stati (osservazioni):
1) posizione del carello
2) velocità del carello 
3) angolo del palo
4) velocità angolare del palo 

Azioni: 
0) spinge il carello a sinistra
1) spinge il carello a destra

Ricompensa:
Se il palo rimane in piedi allora avro una reward +1

Condizioni di fine episodio
1) angolo troppo grande: Il palo cade
2) carello esce dai limiti di pista
3) si supera 500 passi

Obbiettivo: imaprare una politica che muova il carello in modo da non far cadere il palo il più a lungo possibile

In [2]:
obs, info= env.reset()

print(f'Observation after the reset: {obs}')
print(f'information of action {info}')

print("Observation space:", env.observation_space)
print("Action space:", env.action_space)


Observation after the reset: [ 0.0149404  -0.0409965  -0.01206485 -0.03211079]
information of action {}
Observation space: Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)
Action space: Discrete(2)


observation space: [posizione carello, velocità carello, angolo del palo, velocità angolare del palo]
action space: (discreto) [0 sposta a sinistra, 1 sposta a destra]

**Next Things Next**: Now get your `REINFORCE` implementation working on the environment. You can import my (probably buggy and definitely inefficient) implementation here. Or even better, refactor an implementation into a separate package from which you can `import` the stuff you need here. 

**Last Things Last**: My implementation does a **super crappy** job of evaluating the agent performance during training. The running average is not a very good metric. Modify my implementation so that every $N$ iterations (make $N$ an argument to the training function) the agent is run for $M$ episodes in the environment. Collect and return: (1) The average **total** reward received over the $M$ iterations; and (2) the average episode length. Analyze the performance of your agents with these new metrics.

In [10]:
#import numpy as np
#import gymnasium as gym
#import torch 
#import torch.nn as nn
#import torch.optim as optim
#from torch.distributions import Categorical
#import torch.nn.functional as F
#import matplotlib.pyplot as plt
#from torch.utils.tensorboard import SummaryWriter
#import os
#import datetime
#from reinforce_cartpole import PolicyNetwork, ReinforceAgent, TrainAgentRenforce
##import pygame
##_ = pygame.init()
#
#now= datetime.datetime.now()
#data_ora_formattata = now.strftime("%d_%m_%yT%H_%M")
#name= f'run_{data_ora_formattata}'
#
##env = gym.make("CartPole-v1", render_mode="human")
#env = gym.make("CartPole-v1")
##pygame.display.init() 
#name_agent="CartPole_REINFORCE"
#temperature_train=0.7
#general_path= f'Reinforcment_Learning/{name_agent}_{data_ora_formattata}_temp_{temperature_train}'
#
#checkpoint_path=general_path+"/checkpoint"
#bestmodel_path= general_path+"/best_model"
#hyperparamtres_path= general_path+"/hyperparametres"
#
#obs_dim = env.observation_space.shape[0]
#action_dim = env.action_space.n
#
#policy = PolicyNetwork(obs_dim=obs_dim, action_dim=action_dim)
#
#logdir= f'tensorboard/Reinforcment_Learning/{name_agent}/{name}_temp_{temperature_train}'
#
#agent = ReinforceAgent(
#    enviroment=env,
#    logdir=logdir, #da modificare
#    policy=policy,
#    name_agent=name_agent,
#    gamma=0.99,
#    max_lenght=500
#)
#
#trainer = TrainAgentRenforce(
#    reinforcagent=agent,
#    lr=1e-2,
#    num_episode=500,
#    num_episode_validation=10,
#    check_val=10,
#    checkpoint_path=checkpoint_path,
#    best_model_path=bestmodel_path,
#    hyperparams_path=hyperparamtres_path,
#    temperature_train=temperature_train
#)
#
##try:
#running_rewards = trainer.train_agent()
##finally:
##    env.close()
##    pygame.display.quit()
##    pygame.quit()
#
#
##pygame.display.quit()
#
##da fare quella ccosa della finestra pygame e modificarla

## Risultati
Dal codice di partenza fornito è stato pulito e diviso in varie componenti in modo tale da avere una maggiore generalizzazione delle classi.
Successivamente sono state introdotte varie componenti aggiuntive:
1) Un parametro di temperatura che sostanzialmente serve per il campionamento dalla distribuzione di probabilità nella scelta delle azioni.
Più la temperatura è grande e più la distribuzione diventa uniforme e quindi si ha più esplorazione da parte del modello, al contrario più la temperatura è piccola più il modello sceglie azioni deterministiche selezionando sempre quella migliore
2) introduzione di varie metriche: 
    1) Lunghezza dell' episodio: Serve per misurare le performance e la stabilità del modello, infatti è gradevole avere un modello che dopo t iterazioni ha episodi più lunghi fino ad completarli.
    2) Deviazione standard: Valuta la stabilità del modello, se risulta alta oppure altalenante vuol dire che il modello non è del tutto stabile, mentre quando si ha una deviazione standard che tende a diminuire significa che il modello tende a prendere le stesse decisioni.
    3) Min reward: misura la ricompensa minima che si ha nell' episodio, è un ottimo metodo per capire se il modello risulta essere stabile e efficiente all' interno dell' ambiente infatti una volta che il modello impara ci si aspetta una min reward sempre più alta.
    4) max reward: Misura l'efficienza del modello, infatti serve a dare una stima della ricompensa massima ottenuta dal modello 
    5) mean reward: Misura la ricompensa media anche questo da sintomi di stabilità o instabilità infatti, la cosa che ci aspettiamo è che essa salga poichè significherebbe che il modello mediamente compiè azioni che hanno un alta ricompensa.
    6) termination failure/success: Rappresentano il numero di successi/fallimenti all' interno dell' addestramento, questo ci da una misura di quanto il modello poi riesca o meno a portare a termine l'obbiettivo
    7) percentile 10%: serve per valutare gli episodi peggiori che stanno sotto al 10%. ci da una misura di quanto il modello durante l'addestramento riesca a migliorarsi quindi a portare il 10% degli episodi peggiori comunque a delle ottime metriche.

Sono stati fatti vari addestramenti cercando di andare a modificare la temperatura e non introducendo alcuna baseline (essendo esercizio successivo).
Il range di temperatura preso è stato:
1: distribuzione morbida equilibrata tra determinismo e stocasticità 
2: distribuzione quasi uniforme molto piatta quindi qausi solamente esplorazione
0.3-0.5-0.7: distribuzione più deterministica il modello si concentra sul' prendere azioni migliori, poca esplorazione

Il risultato ottenuto è che la temperatura a 0.5 risulti essere effettivamente la migliore anche se porta con se una certa instabilità che si riscontra sia nella reward sia nella deviazione standard. ma comunque è l'unica che riesce a ottenere i risultati migliori

-----
## Exercise 2: `REINFORCE` with a Value Baseline (warm up)

In this exercise we will augment my implementation (or your own) of `REINFORCE` to subtract a baseline from the target in the update equation in order to stabilize (and hopefully speed-up) convergence. For now we will stick to the Cartpole environment.



**First Things First**: Recall from the slides on Deep Reinforcement Learning that we can **subtract** any function that doesn't depend on the current action from the q-value without changing the (maximum of our) objecttive function $J$:  

$$ \nabla J(\boldsymbol{\theta}) \propto \sum_{s} \mu(s) \sum_a \left( q_{\pi}(s, a) - b(s) \right) \nabla \pi(a \mid s, \boldsymbol{\theta}) $$

In `REINFORCE` this means we can subtract from our target $G_t$:

$$ \boldsymbol{\theta}_{t+1} \triangleq \boldsymbol{\theta}_t + \alpha (G_t - b(S_t)) \frac{\nabla \pi(A_t \mid s, \boldsymbol{\theta})}{\pi(A_t \mid s, \boldsymbol{\theta})} $$

Since we are only interested in the **maximum** of our objective, we can also **rescale** our target by any function that also doesn't depend on the action. A **simple baseline** which is even independent of the state -- that is, it is **constant** for each episode -- is to just **standardize rewards within the episode**. So, we **subtract** the average return and **divide** by the variance of returns:

$$ \boldsymbol{\theta}_{t+1} \triangleq \boldsymbol{\theta}_t + \alpha \left(\frac{G_t - \bar{G}}{\sigma_G}\right) \nabla  \pi(A_t \mid s, \boldsymbol{\theta}) $$

This baseline is **already** implemented in my implementation of `REINFORCE`. Experiment with and without this standardization baseline and compare the performance. We are going to do something more interesting.

In [1]:
#import numpy as np
#import gymnasium as gym
#import torch 
#import torch.nn as nn
#import torch.optim as optim
#from torch.distributions import Categorical
#import torch.nn.functional as F
#import matplotlib.pyplot as plt
#from torch.utils.tensorboard import SummaryWriter
#import os
#import datetime
#from reinforce_cartpole import PolicyNetwork, ReinforceAgent, TrainAgentRenforce
##import pygame
##_ = pygame.init()
#
#now= datetime.datetime.now()
#data_ora_formattata = now.strftime("%d_%m_%yT%H_%M")
#name= f'run_{data_ora_formattata}'
#
##env = gym.make("CartPole-v1", render_mode="human")
#env = gym.make("CartPole-v1")
##pygame.display.init() 
#name_agent="CartPole_REINFORCE"
#normalizzation_discount=False
#baseline_discount=False
#temperature_train=[0.3, 0.5, 0.7, 1 , 1.5, 2]
#for temp in temperature_train:
#    for subtract in [True,False]:
#        baseline= "simple_subtract" if subtract else "normalization"
#
#
#        general_path= f'Reinforcment_Learning_Classic_Baseline_sub_std/{name_agent}_{data_ora_formattata}_temp_{temp}_Baseline_{baseline}'
#
#        checkpoint_path=general_path+"/checkpoint"
#        bestmodel_path= general_path+"/best_model"
#        hyperparamtres_path= general_path+"/hyperparametres"
#
#        obs_dim = env.observation_space.shape[0]
#        action_dim = env.action_space.n
#
#        policy = PolicyNetwork(obs_dim=obs_dim, action_dim=action_dim)
#
#        logdir= f'tensorboard/Reinforcment_Learning_Baseline_sub_std/{name_agent}/{name}_temp_{temp}_BaseLine_{baseline}'
#
#        agent = ReinforceAgent(
#            enviroment=env,
#            logdir=logdir, #da modificare
#            policy=policy,
#            gamma=0.99,
#            max_lenght=500
#        )
#
#        trainer = TrainAgentRenforce(
#            reinforcagent=agent,
#            lr=1e-2,
#            num_episode=500,
#            num_episode_validation=10,
#            check_val=10,
#            checkpoint_path=checkpoint_path,
#            best_model_path=bestmodel_path,
#            hyperparams_path=hyperparamtres_path,
#            temperature_train=temp
#        )
#        if baseline:
#            running_rewards = trainer.train_agent(normalizzation_discount=False,baseline_discount_sub=True)
#        else:
#            running_rewards=trainer.train_agent(normalizzation_discount=True,baseline_discount_sub=False)

## Risultati
In questa parte di codice viene implementata la differenza tra due tipi di baseline:
1) Baseline base dove si sottrae ai ritorni la media di essi.
2) Baseline con standardizzazione dove si applica media e varianza dei ritorni.
Questa tecniche servono per la stabilizzazione dei gradietni e dunque ad avere un allenamento più fluido e un modello più stabile.
Queste due tecniche sonoi state trainate con lo stesso range di temperature viste sopra e il risultato ci mostra che non vi è un solo modello che mostra valore migliore.
Si parte con la premessa che dagli esperimenti fatti si nota che la baseline che adotta la normalizzazione riesce ad avere dei gradienti più stabili e diciamo migliori performance in generale.
Ci focalizziamo sopratutto su 4 modelli:
1) temp_0.3_BaseLine_normalization
2) temp_0.5_BaseLine_simple_subtract
3) temp_0.7_BaseLine_normalization
4) temp_0.7_BaseLine_simple_subtract
Possiamo vedere questi risultati nel loro complesso dove la situazione risulta anche diciamo essere un po strana.
Infatti si può vedere come i primi due modelli in Training riportino delle metriche migliori rispetto a gli ultimi due, anche se la situazione non è analoga per la validazione dove si ha quasi la situazione opposta.
Quello che si può tirar fuori dalle metriche che si vedono e l'estrazione di due modelli migliori.
Infatti si può vedere come due modelli spiccano tra tutti il resto che sono:
1) temp_0.3_BaseLine_normalization
2) temp_0.5_BaseLine_simple_subtract
Infatti si può vedere come il loro comportamento risulti abbastanza analogo nei plot delle varie metriche, anche se in termini di stabilità sicuramente risulta esserlo maggiormente il 1 modello essendo che comunica attraverso i grafici una stabilità maggiore sia perchè dopo l'epoca 230 la deviazione standard risulta essere verametnte bassa, e le metriche in generale tendono a esseere molto stabili.
Un' altra piccola differenza di stabilità si vede anche nei grafici di train sia per quanto riguarda la loss sia per quanto riguarda la reward media e ovviamenete dato anche dal maggior numero di successi.






**The Real Exercise**: Standard practice is to use the state-value function $v(s)$ as a baseline. This is intuitively appealing -- we are more interested in updating out policy for returns that estimate the current **value** worse. Our new update becomes:

$$ \boldsymbol{\theta}_{t+1} \triangleq \boldsymbol{\theta}_t + \alpha (G_t - \tilde{v}(S_t \mid \mathbf{w})) \frac{\nabla \pi(A_t \mid s, \boldsymbol{\theta})}{\pi(A_t \mid s, \boldsymbol{\theta})} $$

where $\tilde{v}(s \mid \mathbf{w})$ is a **deep neural network** with parameters $w$ that estimates $v_\pi(s)$. What neural network? Typically, we use the **same** network architecture as that of the Policy.

**Your Task**: Modify your implementation to fit a second, baseline network to estimate the value function and use it as **baseline**. 

In [ ]:
import numpy as np
import gymnasium as gym
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import os
import datetime
from reinforce_cartpole import PolicyNetwork, ReinforceAgent, TrainAgentRenforce
#import pygame
#_ = pygame.init()

now= datetime.datetime.now()
data_ora_formattata = now.strftime("%d_%m_%yT%H_%M")
name= f'run_{data_ora_formattata}'

#env = gym.make("CartPole-v1", render_mode="human")
env = gym.make("CartPole-v1")
#pygame.display.init() 
name_agent="CartPole_REINFORCE"
normalizzation_discount=False
baseline_discount=False
temperature_train=[0.3, 0.5, 0.7, 1 , 1.5, 2]
for temp in [0.5]:

    general_path= f'Reinforcment_Learning_Value_Net/{name_agent}_{data_ora_formattata}_temp_{temp}'

    checkpoint_path=general_path+"/checkpoint"
    bestmodel_path= general_path+"/best_model"
    hyperparamtres_path= general_path+"/hyperparametres"

    obs_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n

    policy = PolicyNetwork(obs_dim=obs_dim, action_dim=action_dim)

    logdir= f'tensorboard/Reinforcment_Learning_Value_Net/{name_agent}/{name}_temp_{temp}'

    agent = ReinforceAgent(
        enviroment=env,
        logdir=logdir, 
        policy=policy,
        gamma=0.99,
        max_lenght=500,
        value_net=True
    )

    trainer = TrainAgentRenforce(
        reinforcagent=agent,
        lr=1e-2,
        num_episode=500,
        num_episode_validation=10,
        check_val=10,
        checkpoint_path=checkpoint_path,
        best_model_path=bestmodel_path,
        hyperparams_path=hyperparamtres_path,
        temperature_train=temp
    )
    running_rewards = trainer.train_agent()

Checkpoint not founded, start a new Training


/data01/dl24natacc/Deep_Learning_Application/Homework_2/reinforce_cartpole.py:88: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  obs= torch.tensor(obs, dtype=torch.float32, device=self.device)
/data01/dl24natacc/Deep_Learning_Application/Homework_2/reinforce_cartpole.py:326: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  returns= torch.tensor(self.reinforceagent.compute_discount_returns(rewards,normalizzation_discount,baseline_discount_sub), device= self.device)


Running reward train from episode 0: 18.0
Average reward test from episode 0: 9.3

Running reward train from episode 10: 9.0
Average reward test from episode 10: 9.3

Running reward train from episode 20: 11.0
Average reward test from episode 20: 9.4

Running reward train from episode 30: 9.0
Average reward test from episode 30: 9.1

Running reward train from episode 40: 10.0
Average reward test from episode 40: 9.7

Running reward train from episode 50: 10.0
Average reward test from episode 50: 9.2

Running reward train from episode 60: 10.0
Average reward test from episode 60: 9.2

Running reward train from episode 70: 10.0
Average reward test from episode 70: 9.3

Running reward train from episode 80: 11.0
Average reward test from episode 80: 8.8

Running reward train from episode 90: 10.0
Average reward test from episode 90: 9.5

Running reward train from episode 100: 11.0
Average reward test from episode 100: 8.9

Running reward train from episode 110: 10.0
Average reward test fro

-----
## Exercise 3: Going Deeper

As usual, pick **AT LEAST ONE** of the following exercises to complete.

### Exercise 3.1: Solving Lunar Lander with `REINFORCE` (easy)

Use my (or even better, improve on my) implementation of `REINFORCE` to solve the [Lunar Lander Environment](https://gymnasium.farama.org/environments/box2d/lunar_lander/). This environment is a little bit harder than Cartpole, but not much. Make sure you perform the same types of analyses we did during the lab session to quantify and qualify the performance of your agents.

### Exercise 3.2: Solving Cartpole and Lunar Lander with `Deep Q-Learning` (harder)

On policy Deep Reinforcement Learning tends to be **very unstable**. Write an implementation (or adapt an existing one) of `Deep Q-Learning` to solve our two environments (Cartpole and Lunar Lander). To do this you will need to implement a **Replay Buffer** and use a second, slow-moving **target Q-Network** to stabilize learning.

### Exercise 3.3: Solving the OpenAI CarRacing environment (hardest) 

Use `Deep Q-Learning` -- or even better, an off-the-shelf implementation of **Proximal Policy Optimization (PPO)** -- to train an agent to solve the [OpenAI CarRacing](https://github.com/andywu0913/OpenAI-GYM-CarRacing-DQN) environment. This will be the most *fun*, but also the most *difficult*. Some tips:

1. Make sure you use the `continuous=False` argument to the environment constructor. This ensures that the action space is **discrete** (we haven't seen how to work with continuous action spaces).
2. Your Q-Network will need to be a CNN. A simple one should do, with two convolutional + maxpool layers, folowed by a two dense layers. You will **definitely** want to use a GPU to train your agents.
3. The observation space of the environment is a single **color image** (a single frame of the game). Most implementations stack multiple frames (e.g. 3) after converting them to grayscale images as an observation.

 